# Clustering and k-NN algorithm

We will find the most similar document given one, using the K-NN algorithm

In [2]:
import turicreate

people = turicreate.SFrame('wik.frame_idx')


### Our data is articles of different people from Wikipedia

In [3]:
people.head()

URI,name,text
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...


## The first thing is to obtain the word count for each one of the Wikipedia articles

In [5]:
people['word_count'] = turicreate.text_analytics.count_words(people['text'])

### We can explore the data with Andre Agassi

In [6]:
agassi = people[people['name']=='Andre Agassi']

Which words would be the most common in Andre Agassi article?

We can use stack() to create a new column with the word counts

In [13]:
agassi_words = agassi.stack('word_count',new_column_name=['word','count'])
agassi_words.sort('count', ascending=False).head()

URI,name,text,word,count
<http://dbpedia.org/resource/Andre_Agassi> ...,Andre Agassi,andre kirk agassi ndre siborn april 29 1970 is an ...,the,42.0
<http://dbpedia.org/resource/Andre_Agassi> ...,Andre Agassi,andre kirk agassi ndre siborn april 29 1970 is an ...,and,17.0
<http://dbpedia.org/resource/Andre_Agassi> ...,Andre Agassi,andre kirk agassi ndre siborn april 29 1970 is an ...,in,16.0
<http://dbpedia.org/resource/Andre_Agassi> ...,Andre Agassi,andre kirk agassi ndre siborn april 29 1970 is an ...,of,14.0
<http://dbpedia.org/resource/Andre_Agassi> ...,Andre Agassi,andre kirk agassi ndre siborn april 29 1970 is an ...,to,14.0
<http://dbpedia.org/resource/Andre_Agassi> ...,Andre Agassi,andre kirk agassi ndre siborn april 29 1970 is an ...,agassi,12.0
<http://dbpedia.org/resource/Andre_Agassi> ...,Andre Agassi,andre kirk agassi ndre siborn april 29 1970 is an ...,slam,9.0
<http://dbpedia.org/resource/Andre_Agassi> ...,Andre Agassi,andre kirk agassi ndre siborn april 29 1970 is an ...,a,8.0
<http://dbpedia.org/resource/Andre_Agassi> ...,Andre Agassi,andre kirk agassi ndre siborn april 29 1970 is an ...,career,8.0
<http://dbpedia.org/resource/Andre_Agassi> ...,Andre Agassi,andre kirk agassi ndre siborn april 29 1970 is an ...,open,7.0


We see that the most common word is "the". This result is not helpful if what we try to do is find a wikipedia article similar to Agassi. So, we will be using term frequency–inverse document frequency (tf-idf). TF-IDF is a statistic which shows how important a word is in a document but taking into account the whole corpus. 

### TD-IDF for the whole corpus

In [16]:
people['tfidf'] = turicreate.text_analytics.tf_idf(people['text'])

In [20]:
agassi = people[people['name']=='Andre Agassi']

In [21]:
agassi[['tfidf']].stack('tfidf',new_column_name=['word','tfidf']).sort('tfidf',ascending=False)


word,tfidf
agassi,98.56688000382495
slam,46.39151620747509
tennis,29.494545967336226
open,22.290002510846538
grand,18.778731576776124
male,17.51397905571643
andre,17.28224614653598
players,15.40895737675724
atrisk,14.595231870222516
atp,13.36486059204205


We see now that the top words are more related to Andre Agassi, with words like "slam" and "tennis.

### Let's see what is the distance between different people

In [33]:
henman = people[people['name']=='Tim Henman']
pacino = people[people['name']=='Al Pacino']
dench = people[people['name']=='Judi Dench']

We will compare Al Pacino and Tim Henman, checking who is closer to Andre Agassi

To measure distance, we will use cosine

In [27]:
turicreate.distances.cosine(agassi['tfidf'][0],pacino['tfidf'][0])

0.9826643021043356

In [34]:
turicreate.distances.cosine(agassi['tfidf'][0],henman['tfidf'][0])

0.7643102444942209

With cosine distance, the farthest distance can be 1. So, we observe that Tim Henman is closer to Andre Agassi than Al Pacino, as expected.

## We build the NN-model

In [35]:
knn_tfidf_model = turicreate.nearest_neighbors.create(people,features=['tfidf'],label='name')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

### We can use it to obtain the closest article to Agassi according to TF-IDF

In [36]:
knn_tfidf_model.query(agassi)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 104.048ms    |

| Done         |         | 100         | 436.503ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Andre Agassi,0.0,1
0,Rafael Nadal,0.7665615141955836,2
0,Daniel Nestor,0.7745098039215687,3
0,Tim Henman,0.7764350453172205,4
0,Grant Connell,0.7914285714285714,5


The closest article is Rafael Nadal, which makes sense since he is also a professional tennis player.

Let's see what article has the closest distance to Al Pacino

In [38]:
knn_tfidf_model.query(pacino)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 42.438ms     |

| 0            | 2       | 0.00338576  | 43.05ms      |

| Done         |         | 100         | 385.416ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Al Pacino,0.0,1
0,Robert Redford,0.8184931506849316,2
0,Ethan Hawke,0.8192771084337349,3
0,Eric McCormack,0.8222222222222222,4
0,Francis Ford Coppola,0.82421875,5


We see Robert Redford is the closest person to Al Pacino.

## k-NN word count model

In [42]:
knn_word_model = turicreate.nearest_neighbors.create(people,features=['word_count'],label='name', distance='cosine')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

We can check the difference between the models by looking to the closest article to Brad Pitt

In [67]:
brad = people[people['name']=='Brad Pitt']

In [64]:
knn_tfidf_model.query(brad)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 186.117ms    |

| 0            | 2       | 0.00338576  | 224.376ms    |

| Done         |         | 100         | 625.848ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Brad Pitt,0.0,1
0,Angelina Jolie,0.7840236686390533,2
0,Alec Baldwin,0.8076923076923077,3
0,Leonardo DiCaprio,0.810126582278481,4
0,Gwyneth Paltrow,0.8114478114478114,5


On this case, we have actor Angelina Jolie as the closest article

In [68]:
knn_word_model.query(brad)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 8.833ms      |

| Done         |         | 100         | 334.604ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Brad Pitt,0.0,1
0,Victor Glynn,0.21424964723862183,2
0,Ben Affleck,0.22371595148268497,3
0,Z%C3%BClf%C3%BC Livaneli,0.2240590295244096,4
0,Larry Woiwode,0.2253853257692532,5


With this model, we have producer Victor Glynn

## In conclusion, the TF-IGF model performs better